In [15]:
from xgboost import XGBClassifier
import pandas as pd
import torch
import pickle
import numpy as np

from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

In [17]:
label_map = {
    'Красота': 0,
    'Политика': 1,
    'Экономика': 2,
    'Мода': 3,
    'Общество': 4,
    'Закон и право': 5,
    'Кино': 6,
    'Телевидение': 7,
    'Персоны': 8,
    'События': 9,
    'Бренды': 10,
    'Наука': 11,
    'Гаджеты': 12,
    'Соцсети': 13,
    'Дом': 14,
    'Технологии': 15,
    'Транспорт': 16,
    'Погода': 17,
    'Рецепты': 18,
    'Опросы': 19,
    'Головоломки': 20
}

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_name = "./drive/MyDrive/VK_hack/xgb_clf.pkl"

xgb_model_loaded = pickle.load(open(file_name, "rb"))

In [43]:
input_text = """В Microsoft показали, как созданная нейросетью Мона Лиза читает рэп
Команда исследователей искусственного интеллекта Research Asia компании Microsoft разработала приложение искусственного интеллекта (ИИ), преобразующее неподвижное изображение человека в правдоподобную анимацию. При наложении звуковой дорожки получившийся цифровой аватар проговорит или пропоет текст с правильной мимикой. Результаты работы опубликованы на портале научных материалов arXiv.
Новую нейросеть назвали VASA-1. Создатели проекта отметили, что ИИ может работать как с фотографиями, так и с рисунками. В качестве демонстрации возможностей группа представила ряд видеороликов, на которых созданные VASA-1 цифровые аватары поют и разговаривают. А «Мону Лизу» Леонардо да Винчи алгоритмы заставили зачитать рэп.
В каждой анимации выражение лица меняется вместе со словами, подчеркивая сказанное. Исследователи также отметили, что, несмотря на реалистичность видео, более пристальное рассмотрение может выявить недостатки и свидетельства того, что они были созданы искусственно.
По словам специалистов, инструмент генерирует видео разрешением 512 на 512 пикселей со скоростью 45 кадров в секунду, а для использования достаточно мощности потребительской видеокарты. Например, создание ролика с помощью графического процессора Nvidia RTX 4090 занимает около двух минут. Команда отметила, что VASA-1 можно использовать для создания дипфейков, поэтому система пока не будет общедоступной."""

input_text = 'Температура и влажность: как они влияют на жизнь населенных пунктов'

In [44]:
tokenized_texts = tokenizer(input_text, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [45]:
with torch.no_grad():
    input_ids = tokenized_texts["input_ids"]
    attention_mask = tokenized_texts["attention_mask"]

    outputs = model(input_ids, attention_mask=attention_mask)

    sentence_embeddings = mean_pooling(outputs, attention_mask).cpu().numpy()

In [46]:
preds = xgb_model_loaded.predict(sentence_embeddings)
mask = [True if i == 1 else False for i in preds[0]]

In [47]:
np.array(list(label_map.keys()))[mask]

array(['Погода'], dtype='<U13')

In [31]:
sum(mask)

0